# Galois LFSRs

In [1]:
start_state = 0b00001
lfsr = start_state
period = 0

while True:
    out = lfsr & 0b1
    print(f"{lfsr:0>5b}", out)
    lfsr >>= 1
    if out:
        lfsr ^^= 0b11100
    if lfsr == start_state:
        break

00001 1
11100 0
01110 0
00111 1
11111 1
10011 1
10101 1
10110 0
01011 1
11001 1
10000 0
01000 0
00100 0
00010 0


# Fibonacci LFSRs

In [2]:
class LFSR:
    def __init__(self, key, taps):
        d = max(taps)
        assert len(key) == d, "Error: key of wrong size."
        self._s = key
        self._t = [d - t for t in taps]

    def _sum(self, L):
        s = 0
        for x in L:
            s ^^= x
        return s

    def _clock(self):
        b = self._s[0]
        self._s = self._s[1:] + [self._sum(self._s[p] for p in self._t)]
        return b

    def getbit(self):
        return self._clock()

fibo_lfsr = LFSR([1, 0, 0, 0, 0], [5, 4, 3])
print(fibo_lfsr._s, fibo_lfsr._t)

[1, 0, 0, 0, 0] [0, 1, 2]


In [3]:
exist_state = set()

for _ in range(16):
    stat = "".join([str(i) for i in fibo_lfsr._s])
    if stat in exist_state:
        break
    exist_state.add(stat)
    print("".join(str(i) for i in reversed(fibo_lfsr._s)), fibo_lfsr.getbit())

00001 1
10000 0
01000 0
00100 0
10010 0
11001 1
11100 0
11110 0
01111 1
10111 1
11011 1
01101 1
00110 0
00011 1


Fibonacci LFSR state `00001` is equivalent to Galois LFSR state `11001`!!!

# Implement Galois LFSRs using companion matrix

In [4]:
P.<x> = PolynomialRing(GF(2))
P = x^5 + x^2 + x + 1 # poly = 0b11100
C = companion_matrix(P, format='left')
C

[0 1 0 0 0]
[0 0 1 0 0]
[1 0 0 1 0]
[1 0 0 0 1]
[1 0 0 0 0]

In [5]:
b = vector([1, 0, 0, 0, 0]) # start_state = 0b00001
exist_state = set()

while True:
    state = "".join(reversed([str(i) for i in b]))
    if state in exist_state:
        break
    print(state)
    b = C * b
    exist_state.add(state)

00001
11100
01110
00111
11111
10011
10101
10110
01011
11001
10000
01000
00100
00010
